In [1]:
import pandas as pd
import numpy as np
import re
import statistics as stat

### Data Cleaning

In [6]:
df = pd.read_csv('WELLCOME_APCspend2013_forThinkful.csv', low_memory = False, encoding = 'iso-8859-1')

# Rename Column Headers
df.rename(columns={'PMID/PMCID': 'pid', 
                   'Publisher': 'pub', 
                   'Journal title': 'j_title', 
                   'Article title':'a_title', 
                   'COST (£) charged to Wellcome (inc VAT when charged)': 'cost'}, 
          inplace=True)

# Common Text Mistakes
df.j_title = df.j_title.str.strip() # Remove whitespaces in journal titles
df.j_title = df.j_title.str.lower() # convert all to uppercase
df.j_title = df.j_title.str.replace(':','')
df.j_title = df.j_title.str.replace('&', 'and')
df.j_title = df.j_title.str.replace(',', '')
df.cost = df.cost.str.strip('$')
df.cost = df.cost.str.strip('£').astype(float)
df = df[df.cost < 9000.00]

# Spelling mistakes
df['j_title'] = df['j_title'].str.replace('antimicobial','antimicrobial')
df['j_title'] = df['j_title'].str.replace('agfents','agents')
df['j_title'] = df['j_title'].str.replace('angewande','angewandte')
df['j_title'] = df['j_title'].str.replace('behaviour','behavior')  
df['j_title'] = df['j_title'].str.replace('ophthalmology','opthalmology')  
df['j_title'] = df['j_title'].str.replace('jnl','journal') 
df['j_title'] = df['j_title'].str.replace('of','') 
df['j_title'] = df['j_title'].str.replace('.','') 
df['j_title'] = df['j_title'].str.replace('neuropathol','neuropathologica') 
df['j_title'] = df['j_title'].str.replace('neuropathologicaogica','neuropathologica') 
df['j_title'] = df['j_title'].str.replace('biinformatics','bionformatics') 
df['j_title'] = df['j_title'].str.replace('journals','journal') 
df['j_title'] = df['j_title'].str.replace('eur j immunol','european journal immunology') 
df['j_title'] = df['j_title'].str.replace('european child and adolescent psychiatty','european child and adolescent psychiatry') 
df['j_title'] = df['j_title'].str.replace('genetic epidemology','genetic epidemiology') 
df['j_title'] = df['j_title'].str.replace('haematologica/the haematology journal','haematologica') 
df['j_title'] = df['j_title'].str.replace('heptology','hepatology')
df['j_title'] = df['j_title'].str.replace('embo journal','embo') 
df['j_title'] = df['j_title'].str.replace('embo reports','embo') 
df['j_title'] = df['j_title'].str.replace('embo molecular medicine','embo') 
df['j_title'] = df['j_title'].str.replace('proceddings','proceedings') 
df['j_title'] = df['j_title'].str.replace('plosone','plos one') 
df['j_title'] = df['j_title'].str.replace('plos  one','plos one') 
df['j_title'] = df['j_title'].str.replace('proc natl acad sci u s a','the national academy sciences') 
df['j_title'] = df['j_title'].str.partition('part')
df['j_title'] = df['j_title'].str.partition('section')
df['j_title'] = df['j_title'].str.partition('online')
df['j_title'] = df['j_title'].str.replace('embo molecular medicine','embo')

# Combine similar journals
df['j_title'].loc[df['j_title'].str.contains('med chem')] = 'journal  medicinal chemistry'
df['j_title'].loc[df['j_title'].str.contains('british journal')] = 'british journal  the history  science'
df['j_title'].loc[df['j_title'].str.contains('clinical nutrition')] = 'american journal for clinical nutrition'
df['j_title'].loc[df['j_title'].str.contains('molecular basis  disease')] = 'biochimica et biophysica acta - molecular basis  disease'

C:\Users\edcha\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
df.j_title.unique()

array(['psychological medicine', 'biomacromolecules',
       'journal  medicinal chemistry', 'j org chem',
       'journal  proteome research', 'mol pharm', 'acs chemical biology',
       'journal  chemical information and modeling', 'biochemistry',
       'gastroenterology', 'journal  biological chemistry',
       'journal  immunology', 'acs chemical neuroscience', 'acs nano',
       'american chemical society', 'analytical chemistry',
       'bioconjugate chemistry', 'journal  the american chemical society',
       'chest', 'journal  neurophysiology', 'journal  physiology',
       'the journal  neurophysiology', 'american journal  psychiatry',
       'americal journal  psychiatry', 'behavioral neuroscience',
       'emotion', 'health psychology', 'journal  abnormal psychology',
       'journal  consulting and clinical psychology',
       'journal  experimental psychology  animal behavior process',
       'journal  experimental psychology human perception and performance',
       'jou

In [8]:
# Top 5 Journals
journal_count = df.groupby('j_title').count()
top5journals = journal_count.sort_values(['a_title'], ascending = False)
top5 = top5journals.head(5).index.tolist()

In [9]:
def journal_stats(journal_list, dataframe):
    for journal in journal_list:
        partition = pd.DataFrame(dataframe[dataframe['j_title']==journal])
        partition.cost.fillna(partition.cost.mean(),inplace=True) # impute na values with mean of that journal        
        partition_mean = partition.cost.mean()
        partition_median = partition.cost.median()
        partition_std = partition.cost.std()
        partition_count = len(partition)
        partition_sum = partition.cost.sum()
        print('Stats for', journal.upper(), '\nTotal: \t\t£{:2.2f}\nMean: \t\t£{:2.2f},\nMedian: \t£{:2.2f},\nStd. Dev: \t£{:2.2f},\nArticle Count: \t{}\n'
              .format(partition_sum, partition_mean, partition_median, partition_std, partition_count))

In [10]:
journal_stats(top5, df)

Stats for PLOS ONE 
Total: 		£177912.97
Mean: 		£931.48,
Median: 	£893.43,
Std. Dev: 	£197.58,
Article Count: 	191

Stats for JOURNAL  BIOLOGICAL CHEMISTRY 
Total: 		£74026.60
Mean: 		£1423.59,
Median: 	£1301.14,
Std. Dev: 	£411.95,
Article Count: 	52

Stats for NEUROIMAGE 
Total: 		£64239.88
Mean: 		£2215.17,
Median: 	£2326.43,
Std. Dev: 	£266.65,
Article Count: 	29

Stats for NUCLEIC ACIDS RESEARCH 
Total: 		£29874.00
Mean: 		£1149.00,
Median: 	£852.00,
Std. Dev: 	£442.94,
Article Count: 	26

Stats for PLOS PATHOGENS 
Total: 		£34603.07
Mean: 		£1572.87,
Median: 	£1600.25,
Std. Dev: 	£161.78,
Article Count: 	22

